In [3]:
# 添加历史对话功能
from __future__ import annotations
import os
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
# 封装Mistral Embedding 
import logging
from typing import Dict, List, Any
# 构建检索问答链
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class MistralAIEmbeddings(BaseModel, Embeddings):
    """`MistralAI Embeddings` embedding models."""

    client: Any
    """`mistralai.MistralClient`"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化MistralClient为values["client"]

        Args:
            values (Dict): 包含配置信息的字典，必须包含 client 的字段.

        Returns:
            values (Dict): 包含配置信息的字典。如果环境中有mistralai库，则将返回实例化的MistralClient类；否则将报错 'ModuleNotFoundError: No module named 'mistralai''.
        """
        from mistralai.client import MistralClient
        api_key = os.getenv('MISTRAL_API_KEY')
        if not api_key:
            raise ValueError("MISTRAL_API_KEY is not set in the environment variables.")
        values["client"] = MistralClient(api_key=api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings(
            model="mistral-embed",
            input=[text]
        )
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]

# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)


template = """使用以下上下文来回答最后的问题。如果你不知道答案或者不确定结果，只需要你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。要求使用中文回答。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 构建检索问答链
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=loaded_db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

# 测试结果
question_1 = "GIS的组成？"
result = qa_chain({"query": question_1})
print(result["result"])


GIS 是由五个主要元素组成：硬件、软件、数据、人员和方法。其中数据是最重要的部件，它可以是地理数据和相关的表格数据，可以从 verschiedenen Quellen, wie z.B. eigenen Erhebungen oder kommerziellen Datenanbietern, erworben werden. GIS-Systeme integrieren diese Daten aus verschiedenen Quellen und können sie mit anderen Datenbankmanagementsystemen verwalten. Die Mitarbeiter sind ebenfalls ein wichtiger Bestandteil von GIS, da sie das System verwalten und für die Anwendung auf reale Probleme planen.


In [5]:
# 添加历史对话功能
from __future__ import annotations
import os
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
# 封装Mistral Embedding 
import logging
from typing import Dict, List, Any
# 构建检索问答链
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class MistralAIEmbeddings(BaseModel, Embeddings):
    """`MistralAI Embeddings` embedding models."""

    client: Any
    """`mistralai.MistralClient`"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化MistralClient为values["client"]

        Args:
            values (Dict): 包含配置信息的字典，必须包含 client 的字段.

        Returns:
            values (Dict): 包含配置信息的字典。如果环境中有mistralai库，则将返回实例化的MistralClient类；否则将报错 'ModuleNotFoundError: No module named 'mistralai''.
        """
        from mistralai.client import MistralClient
        api_key = os.getenv('MISTRAL_API_KEY')
        if not api_key:
            raise ValueError("MISTRAL_API_KEY is not set in the environment variables.")
        values["client"] = MistralClient(api_key=api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings(
            model="mistral-embed",
            input=[text]
        )
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]

# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)


template = """针对以下的问题，请使用两种模式进行回答，A：简明扼要模式，B：详细具体模式；要求使用中文回答。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 构建检索问答链
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=loaded_db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

# 测试结果
question_1 = "GIS是什么"
result1 = qa_chain({"query": question_1})
print(result1["result"])
question_2 = "GIS的应用场景？"
result2 = qa_chain({"query": question_2})
print(result2["result"])


A 简明扼要模式：

GIS（地理信息系统）是一种利用计算机技术管理和应用地球表面空间信息的系统。它是一个有机体，由计算机硬件、软件、地理数据和人员组成，用于分析和提供空间信息，为地理研究和地理决策提供信息来源和技术支持。

B 详细具体模式：

GIS（Geographic Information System，地理信息系统）是一种利用计算机技术来管理和应用地球表面的空间信息的系统。它是一个由计算机硬件、软件、地理数据和人员组成的有机体，可以通过采用地理模型分析方法，适时提供多种空间的和动态的地理信息，为地理研究和地理决策服务的计算机技术系统。GIS 是一种空间型信息系统，主要用于处理地理空间数据，并为用户提供可视化的地理空间信息。

GIS 可以用来存储、显示、处理、分析、输出地理信息，并可以实时更新地理信息。GIS 可以处理各种空间实体及其关系，是城市基础设施之一，也是一种城市现代化管理、规划和科学决策的先进工具。GIS 作为信息技术的一种，是在计算机硬件、软件的支持下，以地理空间数据库为基础，以具有空间内涵的地理数据为处理对象，运用系统工程和信息科学的理论，进行地理信息的采集、存储、显示、处理、分析、输出的计算机系统。
A：简明扼要模式

GIS 的应用场景包括：

1. 解决实际问题，如计算救护车最佳行驶路线、寻找香蕉生长最适合的土壤等。
2. 应用型地理信息系统，如 lis、cgis、ugis 等，解决特定类型地理空间分布问题。
3. 存储和查询地理信息，例如经度、纬度坐标或国家网格坐标等。
4. 提供信息支持，如帮助解决领土争端、设置路标等。
5. 地图制作和地理分析，比传统手工方法更高效和快速。

B：详细具体模式

GIS（地理信息系统）的应用场景非常广泛，主要包括以下几方面：

1. 解决实际问题：GIS 可用于解决各种实际问题，例如：
	* 为新的职业提供信息支持。
	* 寻找香蕉生长最适合的土壤。
	* 计算救护车最佳行驶路线。
	* 记录计划的详细资料。
	* 模拟全球大气循环等。
2. 应用型地理信息系统：应用型地理信息系统是在成熟的工具型 GIS 软件基础上，根据用户的需求和应用目的而设计的地理信息系统。它具有地理空间实体和解决特定地理空间分布问题的模型。例如：
	* lis：用于地理信息化的学习和研究。
	* cgis：用于城市规划和

In [7]:
# 添加历史对话功能
from __future__ import annotations
import os
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
# 封装Mistral Embedding 
import logging
from typing import Dict, List, Any
# 构建检索问答链
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class MistralAIEmbeddings(BaseModel, Embeddings):
    """`MistralAI Embeddings` embedding models."""

    client: Any
    """`mistralai.MistralClient`"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化MistralClient为values["client"]

        Args:
            values (Dict): 包含配置信息的字典，必须包含 client 的字段.

        Returns:
            values (Dict): 包含配置信息的字典。如果环境中有mistralai库，则将返回实例化的MistralClient类；否则将报错 'ModuleNotFoundError: No module named 'mistralai''.
        """
        from mistralai.client import MistralClient
        api_key = os.getenv('MISTRAL_API_KEY')
        if not api_key:
            raise ValueError("MISTRAL_API_KEY is not set in the environment variables.")
        values["client"] = MistralClient(api_key=api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings(
            model="mistral-embed",
            input=[text]
        )
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]

# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)


template = """针对以下的问题，进行作答，不要说多余的话，认真作答即可；要求使用中文回答。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 构建检索问答链
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=loaded_db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

# 测试结果
print("问题：")
question = "遥感的微波的概念是什么"
print(question)
print("模型回答：")
result = qa_chain({"query": question})
print(result["result"])

问题：
遥感的微波的概念是什么
模型回答：
遥感的微波（Microwave Remote Sensing）指的是使用微波辐射的散射和辐射信息来分析、识别地物或提取专题信息的技术。微波遥感可以使用人工辐射源（如微波雷达）发射微波辐射，然后根据地物散射该辐射返回的反向反射信号来探测和测距。由于微波波长比可见光和红外波长长，因此微波遥感具有较好的穿透能力，可以对云层、地表植被、松散沙层和干燥冰雪等物体进行探测，且不受光照等条件的限制，是一种全天候、全天时的遥感技术。微波遥感中常用的微波辐射源波段为0.8~30cm。


In [6]:
# 添加历史对话功能
from __future__ import annotations
import os
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
# 封装Mistral Embedding 
import logging
from typing import Dict, List, Any
# 构建检索问答链
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class MistralAIEmbeddings(BaseModel, Embeddings):
    """`MistralAI Embeddings` embedding models."""

    client: Any
    """`mistralai.MistralClient`"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化MistralClient为values["client"]

        Args:
            values (Dict): 包含配置信息的字典，必须包含 client 的字段.

        Returns:
            values (Dict): 包含配置信息的字典。如果环境中有mistralai库，则将返回实例化的MistralClient类；否则将报错 'ModuleNotFoundError: No module named 'mistralai''.
        """
        from mistralai.client import MistralClient
        api_key = os.getenv('MISTRAL_API_KEY')
        if not api_key:
            raise ValueError("MISTRAL_API_KEY is not set in the environment variables.")
        values["client"] = MistralClient(api_key=api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings(
            model="mistral-embed",
            input=[text]
        )
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]

# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)


template = """针对以下的问题，进行作答，这是多选题，不要说多余的话，从选择题中选择出你的答案；只需要输出选项即可，无需解释其他内容。只能从ABCD中选择对应的字母
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 构建检索问答链
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=loaded_db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})



# 测试结果
print("问题：")
question = "遥感传感器的性能指标有？A：空间分辨率 B：时间分辨率 C:视场角FOV D:光谱分辨率;"
print(question)
print("模型回答：")
result = qa_chain({"query": question})
print(result["result"])

问题：
遥感传感器的性能指标有？A：空间分辨率 B：时间分辨率 C:视场角FOV D:光谱分辨率;
模型回答：
40. 判读标志
42. 时间分辨率
43. 最佳探测波段
51. 传感器的外方位元素
13. 多路、联系并且具有高光谱分辨率方式获取图像的仪器 (could not find an exact match, but closest is option 13)
14. 侧视雷达
3. 斜距投影的性质，平面扫描和圆锥扫描
1. 电磁波
A：空间分辨率
B：时间分辨率
C: 视场角FOV
D: 光谱分辨率


In [7]:
def multi_select_score_v1(true_answer : str, generate_answer : str) -> float:
    # true_anser : 正确答案，str 类型，例如 'BCD'
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(true_answer)
    '''为便于计算，我们假设每道题都只有 A B C D 四个选项'''
    # 先找出错误答案集合
    false_answers = [item for item in ['A', 'B', 'C', 'D'] if item not in true_answers]
    # 如果生成答案出现了错误答案
    for one_answer in false_answers:
        if one_answer in generate_answer:
            return 0
    # 再判断是否全选了正确答案
    if_correct = 0
    for one_answer in true_answers:
        if one_answer in generate_answer:
            if_correct += 1
            continue
    if if_correct == 0:
        # 不选
        return 0
    elif if_correct == len(true_answers):
        # 全选
        return 1
    else:
        # 漏选
        return 0.5


# 测试结果
print("问题：")
question = "遥感传感器的性能指标有？A：空间分辨率 B：时间分辨率 C:视场角FOV D:光谱分辨率;"
print(question)
print("模型回答：")
result = qa_chain({"query": question})
print(result["result"])

score = multi_select_score_v1(true_answer='ABCD', generate_answer=result["result"])
print("模型得分：", score)

问题：
遥感传感器的性能指标有？A：空间分辨率 B：时间分辨率 C:视场角FOV D:光谱分辨率;
模型回答：
40. 判读标志
42. 时间分辨率
43. 最佳探测波段
51. 传感器的外方位元素
13. 多路、联系并且具有高光谱分辨率方式获取图像的仪器 (could not find an exact match, but closest is option 13)
14. 侧视雷达
3. 斜距投影的性质，平面扫描和圆锥扫描
1. 电磁波
A：空间分辨率
B：时间分辨率
C: 视场角FOV
D: 光谱分辨率
模型得分： 1


In [9]:
# 添加历史对话功能
from __future__ import annotations
import os
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
# 封装Mistral Embedding 
import logging
from typing import Dict, List, Any
# 构建检索问答链
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import re
from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class MistralAIEmbeddings(BaseModel, Embeddings):
    """`MistralAI Embeddings` embedding models."""

    client: Any
    """`mistralai.MistralClient`"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化MistralClient为values["client"]

        Args:
            values (Dict): 包含配置信息的字典，必须包含 client 的字段.

        Returns:
            values (Dict): 包含配置信息的字典。如果环境中有mistralai库，则将返回实例化的MistralClient类；否则将报错 'ModuleNotFoundError: No module named 'mistralai''.
        """
        from mistralai.client import MistralClient
        api_key = os.getenv('MISTRAL_API_KEY')
        if not api_key:
            raise ValueError("MISTRAL_API_KEY is not set in the environment variables.")
        values["client"] = MistralClient(api_key=api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings(
            model="mistral-embed",
            input=[text]
        )
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]

# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)


template = """针对以下的问题，进行作答，不要说多余的话，
{context}
问题: {question}

-----------
从选择题中选择出你的答案；只能从ABCD中选择对应的字母;
要求输出的多个选项答案全部包裹在一个```answer```中，例如
```answer
ABC
```，要记住，这是多选题，不要当做单选题
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 构建检索问答链
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=loaded_db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})



def multi_select_score_v1(true_answer : str, generate_answer : str) -> float:
    # true_anser : 正确答案，str 类型，例如 'BCD'
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(true_answer)
    '''为便于计算，我们假设每道题都只有 A B C D 四个选项'''
    # 先找出错误答案集合
    false_answers = [item for item in ['A', 'B', 'C', 'D'] if item not in true_answers]
    # 如果生成答案出现了错误答案
    for one_answer in false_answers:
        if one_answer in generate_answer:
            return 0
    # 再判断是否全选了正确答案
    if_correct = 0
    for one_answer in true_answers:
        if one_answer in generate_answer:
            if_correct += 1
            continue
    if if_correct == 0:
        # 不选
        return 0
    elif if_correct == len(true_answers):
        # 全选
        return 1
    elif if_correct >= len(true_answers):
        # 多选
        return 1
    else:
        # 漏选
        return 0.5


# 正则提取任务
def parse_answer(content): # 从模型生成中字符串匹配提取生成的代码
    pattern = r'```answer(.*?)```'  # 使用非贪婪匹配
    match = re.search(pattern, content, re.DOTALL)
    answer= match.group(1) if match else content
    return answer

# 测试结果
print("问题：")
question = "遥感传感器的性能指标有？A：空间分辨率 B：时间分辨率 C:视场角FOV D:光谱分辨率;"
print(question)
print("模型回答：")
result = qa_chain({"query": question})
print(result["result"])
answer = parse_answer(result["result"])
print("模型答案：",answer)
score = multi_select_score_v1(true_answer='ABCD', generate_answer=answer)
print("模型得分：", score)

问题：
遥感传感器的性能指标有？A：空间分辨率 B：时间分辨率 C:视场角FOV D:光谱分辨率;
模型回答：


Answer: D

Explanation: The ability of a remote sensing sensor to distinguish two different sources of radiation is called radiation resolution. Radiometric correction is the process of correcting systematic and random radiation distortions or deformations caused by external factors during data acquisition and transmission. The vertical distance from a ground point to a reference elevation is called elevation. The new type of sensor that is rapidly developing internationally is a hyperspectral sensor, which is an instrument that obtains images in multiple paths with high spectral resolution.
模型答案： Answer: D

Explanation: The ability of a remote sensing sensor to distinguish two different sources of radiation is called radiation resolution. Radiometric correction is the process of correcting systematic and random radiation distortions or deformations caused by external factors during data acquisition and transmission. The vertical distance from a ground point to a reference elevation is

In [13]:
%pip install jieba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 46.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314457 sha256=016f86e7b063766c0bec138f276d20e40d5d0f5409e5591acc0281f18c17ab49
  Stored in directory: /home/codespace/.cache/pip/wheels/c9/69/31/d56d90b22a1777b0b231e234b00302a55be255930f8bd92dcd
Successfully built jieba

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from nltk.translate.bleu_score import sentence_bleu
import jieba

def bleu_score(true_answer : str, generate_answer : str) -> float:
    # true_anser : 标准答案，str 类型
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(jieba.cut(true_answer))
    # print(true_answers)
    generate_answers = list(jieba.cut(generate_answer))
    # print(generate_answers)
    bleu_score = sentence_bleu(true_answers, generate_answers)
    return bleu_score
true_answer = '微波的的波长比可见光和红外波长，受大气层中云雾的散射影响小，穿透性好，因此能全天候进行遥感。使用微波的遥感被称为微波遥感，它通过接受地面物体发射的微波辐射能量，或接收器本身发出的电磁波束的回波信号，对物体进行探测、识别和分析。由于微波遥感采用主动方式进行，其特点是对云层、地表植被、松散沙层和干燥冰雪具有一定的穿透能力，且不受光照等条线限制。因此微波遥感是一种全天候、全天时的遥感技术。'

print("答案一：")
answer1 = '遥感的微波（Microwave Remote Sensing）指的是使用微波辐射的散射和辐射信息来分析、识别地物或提取专题信息的技术。微波遥感可以使用人工辐射源（如微波雷达）发射微波辐射，然后根据地物散射该辐射返回的反向反射信号来探测和测距。由于微波波长比可见光和红外波长长，因此微波遥感具有较好的穿透能力，可以对云层、地表植被、松散沙层和干燥冰雪等物体进行探测，且不受光照等条件的限制，是一种全天候、全天时的遥感技术。微波遥感中常用的微波辐射源波段为0.8~30cm。'
print(answer1)
score = bleu_score(true_answer, answer1)
print("得分：", score)
print("答案二：")
answer2 = '红外线根据其性质可分为近红外、中红外、远红外和超远红外。近红外主要源于太阳辐射，中红外主要源于太阳辐射及地物热辐射，而远红外和超远红外主要源于地物热辐射。红外波段较宽，在此波段地物间不同的反射特性和发射特性均可较好地表现出来，因此该波段在遥感成像中有着重要作用。在整个红外波段中进行的遥感统称红外遥感，按其内部波段的详细划分可分为近红外遥感和热红外遥感。中红外、远红外和超远红外三者是产生热感的主要原因，因此又称为热红外。热红外遥感最大的特点就是具有昼夜工作的能力。由于摄影胶片感光范围的限制，除了近红外可用于摄影成像外，其他波段不能用于摄影成像，而整个红外波段都可用于扫描成像。'
print(answer2)
score = bleu_score(true_answer, answer2)
print("得分：", score)

Building prefix dict from the default dictionary ...


答案一：
遥感的微波（Microwave Remote Sensing）指的是使用微波辐射的散射和辐射信息来分析、识别地物或提取专题信息的技术。微波遥感可以使用人工辐射源（如微波雷达）发射微波辐射，然后根据地物散射该辐射返回的反向反射信号来探测和测距。由于微波波长比可见光和红外波长长，因此微波遥感具有较好的穿透能力，可以对云层、地表植被、松散沙层和干燥冰雪等物体进行探测，且不受光照等条件的限制，是一种全天候、全天时的遥感技术。微波遥感中常用的微波辐射源波段为0.8~30cm。


Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.752 seconds.
Prefix dict has been built successfully.


得分： 1.0998110061595677e-231
答案二：
红外线根据其性质可分为近红外、中红外、远红外和超远红外。近红外主要源于太阳辐射，中红外主要源于太阳辐射及地物热辐射，而远红外和超远红外主要源于地物热辐射。红外波段较宽，在此波段地物间不同的反射特性和发射特性均可较好地表现出来，因此该波段在遥感成像中有着重要作用。在整个红外波段中进行的遥感统称红外遥感，按其内部波段的详细划分可分为近红外遥感和热红外遥感。中红外、远红外和超远红外三者是产生热感的主要原因，因此又称为热红外。热红外遥感最大的特点就是具有昼夜工作的能力。由于摄影胶片感光范围的限制，除了近红外可用于摄影成像外，其他波段不能用于摄影成像，而整个红外波段都可用于扫描成像。
得分： 9.566325062049814e-232


/home/codespace/.python/current/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/codespace/.python/current/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/codespace/.python/current/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using 